# Problem 1:

Note for no layers, we have 784 inputs going to 10 outputs, and we assume a bias term per each.  This will yield 
$10 \cdot 784 $ parameters, as we have each pixel weighted, and 10 outputs, with a further 10 bias terms, one for each 
output node.  Therefore there is a total of $7840 + 10 = 7850$ parameters when we don't include a hidden layer.

If a model has $k$ hidden layers with $m$ nodes a piece we have the following calculation: For the first layer we have $784m + m$ parameters for intaking from the image to the first layer.  For the next $k-1$ layers, for each layer we have $m^2$ 
interconnects, and $m$ bias terms yielding $(k-1)(m^2 + m)$ parameters.  Finally we have $10m$ connections into the output layer, with an additionally $10$ bias.  Therefore, $$Params(m,k) = 784m + m + (k-1)(m^2 + m) + 10(m+1)$$.

# Problem 2:

For a given number of parameters P, we have the equation $784m + m + (k-1)(m^2 + m) + 10(m+1) = P$, 